In [7]:
import pandas  as pd
from langchain.llms import OpenAI
from dotenv import load_dotenv

import openai,os
load_dotenv(r'D:\Git\NLP\LLM\ActivLoop\.env')
openai_api_key = os.getenv("ACTIVELOOP_TOKEN")

assert openai_api_key, "ERROR: Azure OpenAI Key is missing"
openai.api_key = openai_api_key

openai.api_base = os.getenv("OpenAiService")
openai.api_type = "azure"
openai.api_version =os.getenv("OpenAiVersion")
davincimodel= os.getenv("OpenAiDavinci")
active_loop_token=os.getenv("ACTIVELOOP_TOKEN")
embedding_model=os.getenv("OpenAiEmbedding")
chat_ai=os.getenv("ChatAI")#
HUGGINGFACEHUB_API_TOKEN=os.getenv("HUGGINGFACEHUB_API_TOKEN")

### Introduction
Tools are modular, reusable components meticulously designed to accomplish specific tasks or provide answers to distinct types of questions. By integrating these tools seamlessly into the system, users can effortlessly tap into a diverse range of functionalities and information sources to tackle challenges and generate meaningful responses. In this lesson, we will explore the various Tools in LangChain, uncovering their unique capabilities.

A few notable examples of tools in LangChain, without getting into technical details, are:

1. Google Search: This tool uses the Google Search API to fetch relevant information from the web, which can be used to answer queries related to current events, facts, or any topic where a quick search can provide accurate results.

2. Requests: This tool employs the popular Python library "requests" to interact with web services, access APIs, or obtain data from different online sources. It can be particularly useful for gathering structured data or specific information from a web service.
3. Python REPL: The Python REPL (Read-Eval-Print Loop) tool allows users to execute Python code on-the-fly to perform calculations, manipulate data, or test algorithms. It serves as an interactive programming environment within the LangChain system.
4. Wikipedia: The Wikipedia tool leverages the Wikipedia API to search and retrieve relevant articles, summaries, or specific information from the vast repository of knowledge on the Wikipedia platform.
5. Wolfram Alpha: With this tool, users can tap into the powerful computational knowledge engine of Wolfram Alpha to answer complex questions, perform advanced calculations, or generate visual representations of data.<br><br>
LangChain Agents and Toolkits


In LangChain, an Agent is a bot that acts using natural language instructions and can use tools to answer its queries. Based on user input, it is also used to determine which actions to take and in what order. An action can either be using a tool (such as a search engine or a calculator) and processing its output or returning a response to the user. Agents are powerful when used correctly, as they can dynamically call chains based on user input.<br>

An agent has access to a suite of tools and can decide which of these tools to call, depending on the user input. Tools are functions that perform specific duties. To create an agent in LangChain, you can use the initialize_agent function along with the load_tools function to prepare the tools the agent can use. <br>

For example, you can create a simple agent using the following code. It can use the SerpApi service to fetch Google search results or the Python requests wrapper when required. Remember to install the required packages with the following command: pip install langchain==0.0.208 deeplake openai tiktoken.<br>

In [14]:
llm = OpenAI(engine=davincimodel)


WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [15]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = load_tools(['serpapi', 'requests_all'], llm=llm, serpapi_api_key=os.getenv("SERPAPI_API_KEY"))
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


In [16]:
agent.run("tell me what is midjourney?")



> Entering new  chain...
 this might be a company or a product
Action: Search
Action Input: midjourney
Observation: Midjourney is a generative artificial intelligence program and service created and hosted by San Francisco-based independent research lab Midjourney, Inc. Midjourney generates images from natural language descriptions, called "prompts", similar to OpenAI's DALL-E and Stable Diffusion.
Thought: this is the final answer
Final Answer: Midjourney is a generative artificial intelligence program and service created and hosted by San Francisco-based independent research lab Midjourney, Inc.

> Finished chain.


'Midjourney is a generative artificial intelligence program and service created and hosted by San Francisco-based independent research lab Midjourney, Inc.'

The agent will then use the tool to look for an answer to the query. We’ll see some output examples later in this lesson.

### Google-Search
LLMs inherently possess knowledge only up to the point at which they were trained, leaving them unaware of any information beyond that timeframe. Integrating search engines as tools within the LangChain system presents a significant advantage. The LangChain library provides a Google Search API wrapper that you can use within your project. You can use this wrapper as a standalone utility or as a tool within an agent.

First, make sure you have an API Key and Custom Search Engine ID for the Google Search API. If you don’t already have a Custom Search Engine ID, the following tutorial is a helpful guide for generating one.

https://help.elfsight.com/article/331-how-to-get-search-engine-id<br>
https://support.google.com/googleapi/answer/6158862?hl=en<br>

https://serpapi.com/<br>

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "<YOUR-OPENAI-API-KEY>"
os.environ["GOOGLE_API_KEY"] = "<YOUR-GOOGLE-API-KEY>"
os.environ["GOOGLE_CSE_ID"] = "<YOUR-GOOGLE-CSE-ID>"

# As a standalone utility:
from langchain. utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
search.results("What is the capital of Spain?", 3)

In the LangChain library, using the available tools requires some necessary steps to be taken. First, you need to initialize an agent, which is the central manager for effectively using these tools. Then, we need to define the language model that we want the agent to use.

In [17]:
from langchain.llms import OpenAI

llm = OpenAI(engine=davincimodel, temperature=0)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


Now, we can initialize an agent and load the google-search tool for it to use. The agent will load the search results and provide them to the llm to answer our question. The ZERO_SHOT_REACT_DESCRIPTION type gives the freedom to choose any of the defined tools to provide context for the model based on their description. (You can use different agent types, read more)

In [ ]:
from langchain.agents import initialize_agent, load_tools, AgentType

tools = load_tools(["google-search"])

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

print( agent("What is the national drink in Spain?") )Copy


### Requests
The internet is an extensive source of information that Large Language Models cannot access directly. LangChain offers a convenient wrapper built around the Python Requests module to facilitate seamless interaction between LLMs and this wealth of information. This wrapper accepts a URL as an input and efficiently retrieves data from the specified URL, allowing LLMs to obtain and process web-based content effortlessly.

In this example, we’ll set up a  fake RESTful backend using mockapi.io. To do it, follow these steps:

1. Go to mockapi.io and sign up for a free account.
2. After signing up, log in to your account.
3. Click on "New Project" (the "+" icon) and give your project a name. You don't need to fill in any optional fields.
4. Once the project is created, click on it to view your unique API endpoint.
5. Click on "New Resource" to create a new resource for your API. For example, if you want to create an endpoint for users, you can name the resource "users.”
6. Define the schema for your resource. For instance, if you want each user to have an id, name, and email, you can use the following schema:

{
    "id": "integer",
    "name": "string",
    "email": "string"}

Click on the "Create" button to create the resource with the defined schema.

This fake backend will have an endpoint to retrieve information about fake users stored in the backend. A dictionary will represent each user. For instance:

{
    "id": "1",
    "name": "John Doe",
    "email": "john.doe@example.com"
}

Let's use the LangChain tools to interact with our fake RESTful backend. First, import the necessary libraries and initialize the agent with the desired tools. Then, ask the agent to do an HTTP call at “https://644696c1ee791e1e2903b0bb.mockapi.io/user”: this is the address of our specific mockapi instance, where you should find 30 users. If you want to try your mockapi instance, then replace the address with “https://<YOUR-MOCKAPI-SERVER-ID>.mockapi.io/user”.

In [ ]:
from langchain.agents import AgentType

tools = load_tools(["requests_all"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
  )

response = agent.run("Get the list of users at https://644696c1ee791e1e2903b0bb.mockapi.io/user and tell me the total number of users")

Entering new AgentExecutor chain...

I need to get the content from this URL
Action: requests_get
Action Input: https://644696c1ee791e1e2903b0bb.mockapi.io/user

Observation:

[{"createdAt": "2023-04-24T07:55:47.634Z", "name": "Mr. Kelly Balistreri", "avatar": "https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/1244.jpg", "id": "1"},{"createdAt": "2023-04-24T03:54:44.108Z", "name": "Bradley Cronin", "avatar": "https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/615.jpg", "id": "2"},{"createdAt": "2023-04-24T14:32:29.991Z" , "name": "Jennifer Block Sr.", "avatar": "https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/105.jpg", "id": "3"},

[…]

{"createdAt": "2023-04-24T06:10:38.771Z", "name": "Paula Kshlerin", "avatar": "https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/1145.jpg", "id": "28"},{"createdAt": "2023-04-24T03:15:33.343Z", "name": "Roberto Blanda", "avatar": "https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/575.jpg", "id": "29"},{"createdAt": "2023-04-23T18:20:58.632Z", "name":  "Mr. Lisa Erdman", "avatar": "https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/1172.jpg", "id": "30"}]

Thought:

I now know the list of users
Final Answer: The list of users from the URL https://644696c1ee791e1e2903b0bb.mockapi.io/user is: [{"createdAt": "2023-04-24T07:55:47.634Z", "name": "Mr. Kelly Balistreri", "avatar": "https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/1244.jpg", "id": "1"},{"createdAt": "2023-04-24T03:54:44.108Z", "name": "Bradley Cronin", "avatar": "https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/615.jpg", "id": "2"},{"createdAt": "2023-04-24T14:32:29.991Z", "name":"

[…]

Thought:

I now know the final answer
Final Answer: There are 30 users.

As seen in this example, the agent, initialized with the Request tool and the OpenAI language model, processes the given prompt. It identifies the need to fetch data from the provided URL using a GET request, which is facilitated by the tool of the request. Upon retrieving the user data, the agent analyzes the number of users and returns the result, completing the task.

### Python-REPL
Another tool feature in LangChain is the Python REPL tool, which allows you to execute Python code generated by the language model. This can be useful for complex calculations where the language model generates code to calculate the answer since LLMs are not good at solving algorithmic and math problems. 

Here’s an example of  Python-REPL tool usage:

In [19]:
tools = load_tools(["python_repl"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
  )

print( agent.run("Create a list of random strings containing 4 letters, list should contain 30 examples, and sort the list alphabetically") )



> Entering new  chain...
 I need to create a list of random strings, sort them, and print them out
Action: Python_REPL
Action Input: import random
Observation: 
Thought: I need to create a list of random strings, sort them, and print them out
Action: Python_REPL
Action Input: random_strings = [''.join(random.choices(string.ascii_lowercase, k=4)) for i in range(30)]
Observation: NameError("name 'string' is not defined")
Thought: I need to create a list of random strings, sort them, and print them out
Action: Python_REPL
Action Input: import string
Observation: 
Thought: I need to create a list of random strings, sort them, and print them out
Action: Python_REPL
Action Input: random_strings = [''.join(random.choices(string.ascii_lowercase, k=4)) for i in range(30)]
Observation: 
Thought: I need to sort the list
Action: Python_REPL
Action Input: random_strings.sort()
Observation: 
Thought: I need to print the list
Action: Python_REPL
Action Input: print(random_strings)
Observation: ['am

### Wikipedia
The Wikipedia API tool in LangChain is a powerful tool that allows language models to interact with the Wikipedia API to fetch information and use it to answer questions. Be aware that you need to install the Wikipedia python package using the pip install Wikipedia command. The codes are tested using the 1.4.0 version. However, use the latest version of the libraries.

In [20]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

tools = load_tools(["wikipedia"])

print( agent.run("What is Nostradamus know for") )



> Entering new  chain...
 He is a prophet
Action: Python_REPL
Action Input: print("Nostradamus is a French prophet who lived in the 16th century. He is known for his book Les Propheties, which contains predictions about future events.")
Observation: Nostradamus is a French prophet who lived in the 16th century. He is known for his book Les Propheties, which contains predictions about future events.

Thought: I now know the final answer
Final Answer: Nostradamus is a French prophet who lived in the 16th century. He is known for his book Les Propheties, which contains predictions about future events.

> Finished chain.
Nostradamus is a French prophet who lived in the 16th century. He is known for his book Les Propheties, which contains predictions about future events.


### Wolfram-Alpha
In LangChain, you can integrate Wolfram Alpha by using the WolframAlphaAPIWrapper utility. First, you need to set up a Wolfram Alpha developer account and get your APP ID. 

Wolfram|Alpha APIs: Computational Knowledge Integration
Easily add top-of-the-line computational knowledge into your applications with Wolfram|Alpha APIs. Options from free to pre-built and custom solutions.


products.wolframalpha.com

Wolfram|Alpha APIs: Computational Knowledge Integration
Then, install the Wolfram Alpha Python library with pip install Wolframalpha.
After that, you can set the Wolfram Alpha APP ID as an environment variable in your code:
https://products.wolframalpha.com/api

In [ ]:
import os
os.environ["WOLFRAM_ALPHA_APPID"] = "your_app_id"

from langchain. utilities.wolfram_alpha import WolframAlphaAPIWrapper

wolfram = WolframAlphaAPIWrapper()
result = wolfram.run("What is 2x+5 = -3x + 7?")
print(result)  # Output: 'x = 2/5'Copy


Utilizing the WolframAlphaAPIWrapper from the LangChain library, this code solves the algebraic equation. Let’s observe the WolframAlpha as a tool:

In [ ]:
tools = load_tools(["wolfram-alpha"])

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

print( agent.run("How many days until the next Solar eclipse") )

A Showcase of Resourceful Collaboration
Agents have the capability to employ a variety of tools in order to provide comprehensive and accurate responses to complex inquiries. These agents are better equipped to deliver well-rounded solutions by integrating resources such as Wikipedia and Wolfram-Alpha. Here’s an example of it:

In [24]:
# tools = load_tools(["wolfram-alpha", "wikipedia"], llm=llm)
tools = load_tools(["wikipedia"], llm=llm)
agent = initialize_agent(
		
        tools,
		llm,
		agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
		verbose=True
	)

# agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")
agent.run("Who is the 2022 F1 champion?")



> Entering new  chain...
 I need to find out who the current F1 champion is
Action: Wikipedia
Action Input: "List of Formula One World Drivers' Champions"
Observation: Page: List of Formula One World Drivers' Champions
Summary: Formula One, abbreviated to F1, is the highest class of open-wheeled auto racing defined by the Fédération Internationale de l'Automobile (FIA), motorsport's world governing body. The "formula" in the name refers to a set of rules to which all participants and cars must conform. The Formula One World Championship season consists of a series of races, known as Grands Prix, held usually on purpose-built circuits, and in a few cases on closed city streets. The World Drivers' Championship is presented by the FIA to the most successful Formula One driver over the course of the season through a points system based on individual Grand Prix results. The World Championship is won when it is no longer mathematically possible for another competitor to overtake their poin

'Max Verstappen'

### Conclusion
LangChain agents successfully bring together various tools, like Wikipedia and Wolfram-Alpha, to create a seamless knowledge integration experience. By combining the strengths of these resources, the agents are able to provide clear, accurate, and detailed answers to more complex questions.

Defining custom tools involves creating new classes, functions, or modules that serve specific purposes within your language processing pipeline. These custom tools can enhance or modify existing language processing capabilities provided by the LangChain library or create entirely new functionalities tailored to your specific needs. More on Defining Custom tools will be covered in lessons to come. Happy Learning!

In the next lesson, we’ll build a bot able to refine paragraphs of articles by looking for additional context information online and adding it to that paragraph.